In [1]:
import cv2, time, warnings
import numpy as np
from pynq.lib.video import *
warnings.filterwarnings('ignore')
import pynq
from pynq.overlays.base import BaseOverlay
from pynq import Overlay
from pynq import allocate

overlay = BaseOverlay('base_w_sobel.bit')

sobel = overlay.sobel_0
dma = overlay.axi_dma_0

def reset_all_dma():
    def dma_reset(dma):
        dma.sendchannel.stop()
        dma.recvchannel.stop()
        dma.sendchannel.start()
        dma.recvchannel.start()
    dma_reset(dma)
reset_all_dma()

resolution=(640,480)
padd=int((max(resolution)-min(resolution))/2)
recive_head=0+padd
recive_tail=recive_head+min(resolution)
buffer_shape=max(resolution)

input_buffer = allocate(shape=(buffer_shape,buffer_shape), dtype=np.uint8)
output_buffer = allocate(shape=(buffer_shape,buffer_shape), dtype=np.uint8)

def run_sobel():
    reset_all_dma()
    dma.sendchannel.transfer(input_buffer)
    dma.recvchannel.transfer(output_buffer)
    sobel.write(0x10,buffer_shape)
    sobel.write(0x18,buffer_shape)
    sobel.write(0x00,0x81) #start訊號
    dma.sendchannel.wait()
    dma.recvchannel.wait()

frame_in_w,frame_in_h = resolution
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
cap.isOpened()

True

In [3]:
'''
With PL accelerate
'''
frame_out_w,frame_out_h = resolution
Mode = VideoMode(frame_out_w,frame_out_h,8)
hdmi_out = overlay.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_GRAY)
hdmi_out.start()

start=time.time()
num_frames=0
readError=0
while (overlay.buttons[3].read()==0):
    fstime=time.time()
    frame, image=cap.read()
    if (frame):
        #image=cv2.GaussianBlur(image,(3,3),0)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        image= cv2.copyMakeBorder(image,padd,padd,0,0,cv2.BORDER_CONSTANT,value=[0,0,0])
        input_buffer[:,:] = image
        run_sobel()
        outframe=hdmi_out.newframe()
        outframe=output_buffer[recive_head:recive_tail,:]
        cv2.putText(outframe,"FPS:"+str(round(1/(time.time()-fstime),4)),(1,20),0,0.8,(255,255,255),1)
        hdmi_out.writeframe(outframe)
        num_frames+=1
    else:
        readError+=1
end=time.time()
print("Break signal...")
print("Frames per second: " + str((num_frames-readError) / (end - start)))
print("Number of read errors: " + str(readError))

Break signal...
Frames per second: 23.001548488679344
Number of read errors: 0


In [4]:
'''
Using OpenCV (without PL accelerate)
'''
frame_out_w,frame_out_h = resolution
Mode = VideoMode(frame_out_w,frame_out_h,24)
hdmi_out = overlay.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

start=time.time()
num_frames=0
readError=0
while (overlay.buttons[3].read()==0):
    fstime=time.time()
    ret, frame_vga = cap.read()
    if (ret):
        outframe = hdmi_out.newframe()
        laplacian_frame = cv2.Laplacian(frame_vga, cv2.CV_8U,ksize=3, dst=outframe)
        cv2.putText(outframe,"FPS:"+str(round(1/(time.time()-fstime),4)),(10,20),0,0.8,(255,255,255),1)
        hdmi_out.writeframe(outframe)
        num_frames+=1
    else:
        readError += 1
end = time.time()
#hdmi_out.close()
print("Break signal...")
print("Frames per second: " + str((num_frames-readError) / (end - start)))
print("Number of read errors: " + str(readError))

Break signal...
Frames per second: 9.767277081689846
Number of read errors: 0
